In [6]:
from pwn import remote, context
context.log_level = "error"

def query(mangle_in):
    r = remote("the-other-css.chal.pwni.ng", 1996)
    # challenge key is null, mangle is low bits set
    r.sendline(bytes([0] * 8 + list(mangle_in)))
    res = r.recv(8)
    r.close()
    return list(res)

mangle_ins = [
    [0, 0, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 1],
]

mangle_outs = [query(mangle_in) for mangle_in in mangle_ins]

for mangle_out in mangle_outs:
    print(mangle_out) 

[240, 150, 0, 62, 234, 110, 177, 181]
[236, 247, 121, 13, 52, 60, 10, 181]
[240, 27, 101, 225, 163, 29, 157, 148]
[6, 96, 115, 108, 181, 159, 107, 22]
[16, 84, 34, 56, 198, 203, 125, 52]
[231, 148, 41, 2, 11, 205, 176, 200]
[44, 247, 96, 97, 104, 28, 81, 25]
[181, 239, 145, 180, 153, 58, 200, 107]
[7, 186, 235, 63, 143, 58, 177, 204]


In [7]:
import z3
from css.table import table

def mix(key, value):
    ret = value ^ z3.LShR(value, 8) ^ key
    return ret

def shift(value):
    ret = value ^ (value << 56)
    return ret

def build_tabulate_one(solver):
    tabulate_one = z3.Function("tabulate", z3.BitVecSort(8), z3.BitVecSort(8))
    for idx, table_i in enumerate(table):
        solver.add(tabulate_one(idx) == table_i)
    return tabulate_one

def tabulate(value, name, tabulate_one, solver):
    value_sym = z3.BitVec(name, 64)
    solver.add(value_sym == value)
    ret = []
    for pos in reversed(range(0, 64, 8)):
        ret.append(tabulate_one(z3.Extract(pos+7, pos, value_sym)))
    ret = z3.Concat(*ret)
    return ret

def u8s_to_bitecval(x):
    return z3.BitVecVal(int(bytes(x).hex(), 16), len(x) * 8)

In [8]:
s = z3.Solver()

tabulate_one = build_tabulate_one(s)

key = z3.BitVec("key", 64)

for idx, (mangle_in, mangle_out) in enumerate(zip(mangle_ins, mangle_outs)):
    value = u8s_to_bitecval(mangle_in)
    goal = u8s_to_bitecval(mangle_out)

    value = mix(key, value)
    value = shift(value)
    value = mix(key, value)
    value = shift(value)
    value = mix(key, value)
    value = tabulate(value, f"one_{idx}", tabulate_one, s)
    value = shift(value)
    value = mix(key, value)
    value = tabulate(value, f"two_{idx}", tabulate_one, s)
    value = shift(value)
    value = mix(key, value)
    value = shift(value)
    value = mix(key, value)

    s.add(value == goal)

print(s.check())

cipher_auth_key = hex(s.model()[key].as_long())[2:].zfill(16)
print(cipher_auth_key)

sat
0bfb91847347be4a


In [9]:
from css.mangle import mangle

def bxor(a, b):
    return bytes([i ^ j for i, j in zip(a, b)])
def do_cipher_auth_key(x):
    k = bytes.fromhex(cipher_auth_key)
    return bxor(x, k)

r = remote("the-other-css.chal.pwni.ng", 1996)

host_challenge = bytes([0] * 16)
r.send(host_challenge)
challenge_key = host_challenge[:8]
encrypted_host_nonce = host_challenge[8:]
host_mangling_key = do_cipher_auth_key(challenge_key)

r.recv(8)

host_nonce = do_cipher_auth_key(encrypted_host_nonce)

player_challenge_key = r.recv(8)
encrypted_player_nonce = r.recv(8)
player_nonce = do_cipher_auth_key(encrypted_player_nonce)

player_mangling_key = do_cipher_auth_key(player_challenge_key)
response = mangle(player_mangling_key, do_cipher_auth_key(player_nonce))
r.send(response)

mangling_key = bxor(host_mangling_key, player_mangling_key)
session_nonce = bxor(host_nonce, player_nonce)
session_key = mangle(mangling_key, session_nonce)

In [10]:
from tqdm import tqdm
from itertools import count, cycle
from css.cipher import Cipher
from css.mode import Mode

sectors = []
for _ in tqdm(count()):
    ct = b""
    try:
        ct = r.recvn(8208, timeout=5)
        sectors.append(ct)
    except Exception:
        ct = r.recv()
        sectors.append(ct)
    if len(ct) != 8208:
        break

stream_cipher = Cipher(session_key, Mode.Data)
print(session_key.hex())

decrypted_sectors = []
for sector in tqdm(sectors):
    sector = stream_cipher.decrypt(sector)
    x, t = sector[:16], sector[16:]
    decrypted_sectors.append(bxor(cycle(x), t))

with open("diskA.iso", "wb") as f:
    for s in decrypted_sectors:
        f.write(s)

729it [01:14,  9.83it/s]


042d21becf5bb0e3


100%|██████████| 730/730 [01:25<00:00,  8.49it/s]
